In [1]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 38.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 53.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 39.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.38.0-py3-none-any.whl size=25319964 sha256=6a9d

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import string
import re
from langdetect import detect
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import ktrain
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [4]:
df_dota = pd.read_csv("/kaggle/input/gosuai-dota-2-game-chats/dota2_chat_messages.csv")

In [5]:
df_league_of_legends = pd.read_csv("/kaggle/input/league-of-legends-tribunal-chatlogs/chatlogs.csv")

Getting column number of column containing chat text in both dataset

In [6]:
df_dota.columns.get_loc('text')

3

In [7]:
df_league_of_legends.columns.get_loc('message')

1

In [8]:
df_train = pd.DataFrame(columns = ["chat", "sentiment"])

In [9]:
df_train["chat"] = pd.concat([df_dota[df_dota.columns[3]], df_league_of_legends[df_league_of_legends.columns[1]]], ignore_index=True)

In [10]:
df_train = df_train.sample(n=70000)

In [11]:
len(df_train)

70000

Removing digits, punctuation, URL and email.

In [12]:
def remove_digits(text):
    return ''.join([char for char in str(text) if not char.isdigit()])

def remove_punctuation(text):
    return ''.join([char for char in str(text) if char not in string.punctuation])

def remove_email_address(text):
    return re.sub("\S*@\S*\s?", "", str(text))

def remove_url(text):
    return re.sub("http\S+", "", str(text))

def preprocess_text(dataframe_column):
    dataframe_column = dataframe_column.map(lambda text : remove_digits(text))
    dataframe_column = dataframe_column.map(lambda text : remove_punctuation(text))
    dataframe_column = dataframe_column.map(lambda text : remove_email_address(text))
    dataframe_column = dataframe_column.map(lambda text : remove_url(text))
    return dataframe_column

In [13]:
df_train['chat'] = preprocess_text(df_train['chat'])

In [14]:
df_train = df_train[df_train['chat'].notna()]

No null row now

In [15]:
df_train['chat'].describe()

count     70000
unique    44137
top            
freq       3703
Name: chat, dtype: object

In [16]:
df_train = df_train.drop_duplicates(subset=['chat'], keep='first')

In [17]:
df_train['chat'].describe()

count                 44137
unique                44137
top       мы ща чпокнем вас
freq                      1
Name: chat, dtype: object

In [18]:
df_train['chat'].replace('', np.nan, inplace=True)

In [19]:
df_train = df_train.dropna(subset = ['chat']).reset_index(drop=True)

In [20]:
df_train.describe()

,chat,sentiment
count,44136,0
unique,44136,0
top,мы ща чпокнем вас,NaN
freq,1,NaN


In [21]:
df_train.reset_index(drop=True, inplace = True)

In [22]:
df_train.describe()

,chat,sentiment
count,44136,0
unique,44136,0
top,мы ща чпокнем вас,NaN
freq,1,NaN


In [23]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [24]:
def build_row(row):
    compound_score = sentiment_analyzer.polarity_scores(str(row['chat']))['compound']
    if compound_score <= -0.05:
        senti_score = 0
    elif compound_score > -0.05 and compound_score < 0.05:
        senti_score = 1
    elif compound_score >= 0.05:
        senti_score = 2
    return senti_score

In [25]:
df_train['sentiment'] = df_train.apply(lambda row: build_row(row), axis = 1)

In [26]:
x = df_train['chat'].values.tolist()
y = df_train['sentiment'].values.tolist()

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                   random_state=42,  
                                   test_size=0.33,  
                                   shuffle=True) 

In [28]:
len(x_train), len(y_train), len(x_test), len(y_test)

(29571, 29571, 14565, 14565)

In [29]:
x_train = list(x_train)
y_train = list(y_train)
x_test = list(x_test)
y_test = list(y_test)

In [30]:
(x_train,  y_train), (x_test, y_test), preproc = ktrain.text.texts_from_array(x_train=x_train, y_train=y_train,                                                                    
                                                                              x_test=x_test, y_test=y_test,                                                                              
                                                                              class_names=['0','1', '2'],
                                                                              maxlen=128,
                                                                              preprocess_mode='bert')

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: ru


Is Multi-Label? False
preprocessing test...
language: ru


task: text classification


In [31]:
model = ktrain.text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=32)

Is Multi-Label? False
maxlen is 128


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [32]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  91812096    ['Input-Token[0][0]']            
 g)                              (119547, 768)]                                                   
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']    

In [33]:
result = learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
925/925 [==============================] - 826s 867ms/step - loss: 0.3028 - accuracy: 0.8879
Epoch 2/4
925/925 [==============================] - 806s 871ms/step - loss: 0.1123 - accuracy: 0.9646
Epoch 3/4
925/925 [==============================] - 805s 870ms/step - loss: 0.0580 - accuracy: 0.9819
Epoch 4/4
925/925 [==============================] - 805s 870ms/step - loss: 0.0207 - accuracy: 0.9938


In [34]:
validate = learner.validate(val_data=(x_test, y_test), class_names=['0', '1', '2'])

456/456 [==============================] - 138s 293ms/step
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1457
           1       0.99      0.99      0.99     11383
           2       0.96      0.96      0.96      1725

    accuracy                           0.98     14565
   macro avg       0.96      0.97      0.97     14565
weighted avg       0.98      0.98      0.98     14565



In [35]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [36]:
predictor.get_classes()

['0', '1', '2']